# Исследование надежности заемщиков


**Описание проекта**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных**

* `children` — количество детей в семье  
* `days_employed` — общий трудовой стаж в днях  
* `dob_years` — возраст клиента в годах  
* `education` — уровень образования клиента  
* `education_id` — идентификатор уровня образования  
* `family_status` — семейное положение  
* `family_status_id` — идентификатор семейного положения  
* `gender` — пол клиента  
* `income_type` — тип занятости  
* `debt` — имел ли задолженность по возврату кредитов  
* `total_income` — ежемесячный доход  
* `purpose` — цель получения кредита  

## Импорт библиотек и загрузка данных

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Данные загружены. Наблюдаем пропуски и несоответствие типов данных в некоторых столбцах. Подготовим данные к работе.

## Предобработка данных

### Исследование пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
pd.DataFrame(data.isna().mean()*100).style.format(precision=1).background_gradient('coolwarm')

,0
children,0.0
days_employed,10.1
dob_years,0.0
education,0.0
education_id,0.0
family_status,0.0
family_status_id,0.0
gender,0.0
income_type,0.0
debt,0.0


На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в столбце `total_income` медианным значением по каждому типу из столбца `income_type`.
Пропуски в столбце `days_employed` обработаем позже.

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [8]:
data['total_income'].isna().sum()

0

### Обработка аномальных значений

Заменим все отрицательные значения столбца `days_employed` положительными.

In [9]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [10]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения.

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [12]:
round(len(data[(data['children'] == -1) | (data['children'] == 20)])/len(data)*100, 2)

0.57

Удалим строки, в которых встречаются аномальные значения в столбце `children`, их доля составляет менее 1%

In [13]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [14]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями для каждого типа занятости `income_type`.

In [15]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [16]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропуски обработаны.

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [17]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [18]:
data['education'] = data['education'].str.lower()

In [19]:
data.duplicated().sum()

71

In [20]:
data = data.drop_duplicates()

In [21]:
data.duplicated().sum()

0

Дубликаты обработаны.

### Категоризация данных

Создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [23]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [25]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [26]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

Проведена категоризация данных.

### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [27]:
data['children'].value_counts()

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Ввиду небольшого процента заемщиков с 2 и более детьми логично объединить их в отдельную группу: 2 и более детей.
Категории заемщиков по количеству детей:
0 - 0 детей;
1 - 1 ребенок;
2 - 2 и более детей.

In [28]:
def children_category(df):
    try:
        if df == 0:
            return 0
        if df == 1:
            return 1
        if df >= 2:
            return 2
    except:
        pass

In [29]:
data['children_category'] = data['children'].apply(children_category)

In [30]:
children_debt = data.pivot_table(index=['children_category'], columns=['debt'], values=['purpose'], aggfunc='count')
children_debt.columns = ['no_debt', 'debt']
children_debt['share_of_debtors, %'] = (children_debt['debt']/(children_debt['debt']+children_debt['no_debt'])*100).round(2)
children_debt

,no_debt,debt,"share_of_debtors, %"
children_category,,,
0,13028,1063,7.54
1,4364,444,9.23
2,2207,225,9.25


**Вывод:** Среди заемщиков не имеющих детей должников меньше почти на 2%, по сравнению с клиентами с детьми.

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
family_status_debt = data.pivot_table(index=['family_status'], columns=['debt'], values=['purpose'], aggfunc='count')
family_status_debt.columns = ['no_debt', 'debt']
family_status_debt['share_of_debtors, %'] = (family_status_debt['debt']/ \
                                             (family_status_debt['no_debt']+family_status_debt['debt'])*100).round(2)
family_status_debt

,no_debt,debt,"share_of_debtors, %"
family_status,,,
Не женат / не замужем,2523,273,9.76
в разводе,1105,84,7.06
вдовец / вдова,888,63,6.62
гражданский брак,3749,385,9.31
женат / замужем,11334,927,7.56


**Вывод:** Меньше всего должников среди овдовевших 6.6% и разведенных клиентов 7.1%. Чуть хуже показатели среди клиентов состоящих в браке: задолженность имеет 7.6%. Больше всего должников среди клиентов, состоящих в гражданском браке 9.3% и холостых 9.8%.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
data['total_income_category'].value_counts()

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64

Достаточное количество данных для сравнения имеется только для групп B и C.

In [33]:
family_debt = data.pivot_table(index=['total_income_category'], columns=['debt'], values=['purpose'], aggfunc='count')
family_debt.columns = ['no_debt', 'debt']
family_debt['share_of_debtors, %'] = (family_debt['debt']/(family_debt['no_debt'] + family_debt['debt'])*100).round(2)
family_debt

,no_debt,debt,"share_of_debtors, %"
total_income_category,,,
A,23,2,8.00
B,4660,354,7.06
C,14568,1353,8.50
D,328,21,6.02
E,20,2,9.09


**Вывод:** Процент должников в группе заемщиков с более высоким доходом ниже на 1,5%. 

#### Как разные цели кредита влияют на его возврат в срок?

In [34]:
data['purpose_category'].value_counts()

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64

Группы обладают достаточным количеством данных для их сравнения.

In [35]:
purpose_debt = data.pivot_table(index=['purpose_category'], columns=['debt'], values=['purpose'], aggfunc='count')
purpose_debt.columns = ['no_debt', 'debt']
purpose_debt['share_of_debtors, %'] = (purpose_debt['debt']/(purpose_debt['no_debt'] + purpose_debt['debt'])*100).round(2)
purpose_debt

,no_debt,debt,"share_of_debtors, %"
purpose_category,,,
операции с автомобилем,3879,400,9.35
операции с недвижимостью,9971,780,7.26
получение образования,3619,369,9.25
проведение свадьбы,2130,183,7.91


**Вывод:** Самые "рисковые" группы клиентов те, которые берут кредит на автомобиль или образование. Вероятность наличия просрочек по платежам у клиентов, берущих займ для проведения свадьбы и операций с недвижимостью ниже примерно на 1,5% и 2% соответственно.

### Общий вывод.

**Исходные данные:** таблица с общим количеством строк 21525.

В ходе предобработки данных выявлено 2174 строки с пропущенными данными в столбцах с трудовым стажем и общим доходом. Пропуски обусловлены техническими ошибками или человеческим фактором. Пропущенные значения были заполнены медианными значениями для каждого типа занятости.

**В ходе исследования данных были выявлены следующие зависимости:**

* Клиенты банка не иеющие детей реже имеют задолженности (7,5% против 9,2%)
* Вдовцы и разведенные клиенты реже имеют задолженности (6,6% и 7,1% соответственно), чем холостые и люди, живущие в гражданском браке (9,8% и 9,3%). Семейные люди имеют задолженности немного чаще (7,6%), чем вдовцы и разведенные.
* Заемщики с более высоким доходом реже становятся должниками. Для сравнения были выделены группы с доходом от 50 т.р. до 200 т.р (8,5%) и от 200 т.р до 1 млн.р. (7,1%)
* Реже становятся должниками люди, берущие кредит на операции с недвижимостью (7,3%) и проведение свадьбы (7,9%). Чаще - на операции с автомобилем (9,4%) и получение образования (9,25%)